In [1]:
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import math
import seaborn as sb
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torchvision.transforms import transforms as T
from torch import optim
from tqdm import tqdm
import torch.utils.data as data_utils
import matplotlib.pyplot as plt

In [2]:
data1 = pd.read_csv("podaci_FF_popunjeno.csv")

In [3]:
data1.head()

,DateTime,T,P,U,Ff,Nh,H,VV,sss,"DD_Calm, no wind",...,DD_Wind blowing from the north-west,DD_Wind blowing from the south,DD_Wind blowing from the south-east,DD_Wind blowing from the south-southeast,DD_Wind blowing from the south-southwest,DD_Wind blowing from the south-west,DD_Wind blowing from the west,DD_Wind blowing from the west-northwest,DD_Wind blowing from the west-southwest,DD_variable wind direction
0,2012-09-26 08:00,21.0,1013.9,3,9.417657,0,NaN,NaN,0.0,0,...,0,0,0,0,0,0,0,0,1,0
1,2012-09-26 09:00,24.0,1013.9,4,8.170301,0,NaN,NaN,0.0,0,...,0,0,0,0,1,0,0,0,0,0
2,2012-09-26 10:00,26.0,1013.9,4,7.627709,0,NaN,NaN,0.0,0,...,0,0,0,0,1,0,0,0,0,0
3,2012-09-26 11:00,28.0,1013.9,3,7.189949,0,NaN,NaN,0.0,0,...,0,0,0,0,0,1,0,0,0,0
4,2012-09-26 12:00,29.0,1013.9,4,7.513517,0,NaN,NaN,0.0,0,...,0,1,0,0,0,0,0,0,0,0


In [4]:
class FillModel(nn.Module):
    def __init__(self):
        super(FillModel, self).__init__()
                    
        self.fnn = nn.Sequential(
            nn.Linear(24, 64),
            nn.ReLU(),
            nn.Dropout(p = 0.2),
            
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(p = 0.2),
            
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Dropout(p = 0.2),

            nn.Linear(32, 1),
            nn.ReLU(),
        )
        
    def forward(self, input):
        return self.fnn(input)

In [5]:
model = FillModel()
model.load_state_dict(torch.load("modelVV"))
model.eval()

FillModel(
  (fnn): Sequential(
    (0): Linear(in_features=24, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=32, out_features=1, bias=True)
    (10): ReLU()
  )
)

In [6]:
data2 = data1.drop(["H"], axis = 1)

In [7]:
data2.head()

,DateTime,T,P,U,Ff,Nh,VV,sss,"DD_Calm, no wind",DD_Wind blowing from the east,...,DD_Wind blowing from the north-west,DD_Wind blowing from the south,DD_Wind blowing from the south-east,DD_Wind blowing from the south-southeast,DD_Wind blowing from the south-southwest,DD_Wind blowing from the south-west,DD_Wind blowing from the west,DD_Wind blowing from the west-northwest,DD_Wind blowing from the west-southwest,DD_variable wind direction
0,2012-09-26 08:00,21.0,1013.9,3,9.417657,0,NaN,0.0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2012-09-26 09:00,24.0,1013.9,4,8.170301,0,NaN,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2012-09-26 10:00,26.0,1013.9,4,7.627709,0,NaN,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,2012-09-26 11:00,28.0,1013.9,3,7.189949,0,NaN,0.0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2012-09-26 12:00,29.0,1013.9,4,7.513517,0,NaN,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
data_vv_nan = [x[1:] for x in data2.values if x[6] != x[6]]

In [9]:
data_vv_nan

[array([21.0, 1013.9, 3, 9.417656898498535, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=object),
 array([24.0, 1013.9, 4, 8.17030143737793, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=object),
 array([26.0, 1013.9, 4, 7.62770938873291, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=object),
 array([28.0, 1013.9, 3, 7.189949035644531, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=object),
 array([29.0, 1013.9, 4, 7.513517379760742, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object),
 array([29.0, 1013.9, 3, 8.013284683227539, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=object),
 array([30.0, 1012.9, 2, 6.997527122497559, 0, nan, 0.0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], dtype=object),
 array([30.0, 1012.9, 3, 6.892694473266602, 0, nan, 0.0, 

In [10]:
data_vv_nan = [[*x[:5], *x[6:]] for x in data_vv_nan]

In [11]:
data_vv_nan = torch.Tensor(data_vv_nan)

In [12]:
with torch.no_grad():
    output = model(data_vv_nan)

In [13]:
output = np.array(output.squeeze(1))

In [14]:
index = 0
for (i, x) in enumerate(data1["VV"]):
    if(x != x):
        data1["VV"][i] = output[index]
        index+=1 

<ipython-input-14-e55cdef8961a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1["VV"][i] = output[index]


In [15]:
data1.isna().sum()

DateTime                                        0
T                                               0
P                                               0
U                                               0
Ff                                              0
Nh                                              0
H                                           53192
VV                                              0
sss                                             0
DD_Calm, no wind                                0
DD_Wind blowing from the east                   0
DD_Wind blowing from the east-northeast         0
DD_Wind blowing from the east-southeast         0
DD_Wind blowing from the north                  0
DD_Wind blowing from the north-east             0
DD_Wind blowing from the north-northeast        0
DD_Wind blowing from the north-northwest        0
DD_Wind blowing from the north-west             0
DD_Wind blowing from the south                  0
DD_Wind blowing from the south-east             0


In [16]:
data1.to_csv("podaci_VV_popunjeno.csv", index= False)